In [16]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model
import tensorflow as tf
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, SpatialDropout1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer

from keras import callbacks
from keras.callbacks import CSVLogger

import os

#traindata = pd.read_csv('preprocessed_train_multiclass(a)_new.csv', header=None)
#testdata = pd.read_csv('preprocessed_test_multiclass(a)_new.csv', header=None)
'''
X=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli121.csv", header=None )
T=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli11.csv", header=None )
C=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1_label.csv",header=None)
Y=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2_label.csv",header=None)
X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain_1.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\train_label.csv", header=None)

T = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_1.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_label.csv", header=None)'''

X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\sptrain1_t1_1.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\t1_2_label.csv", header=None)

T = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\sptrain1_t2-1.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\tr1\t2_2_label.csv", header=None)

X1 = X.iloc[0:,0:75]# 0 to 73 columns of data frame with all rows
T1 = T.iloc[0:,0:75]# 
X1.fillna(0,inplace=True)
X1.replace([np.inf,-np.inf],1,inplace=True)
T1.fillna(0,inplace=True)
T1.replace([np.inf,-np.inf],1,inplace=True)
#X1.value_counts('Label')
'''
for index in X1.columns:
    X1[index]=X1[index].fillna(0,inplace=True)
    X1[index]=X1[index].replace([np.inf,-np.inf],1,inplace=True)
'''

scaler = Normalizer().fit(X1)
trainX = scaler.transform(X1)


scaler = Normalizer().fit(T1)
testT = scaler.transform(T1)

y_train1 = np.array(Y)
y_test1 = np.array(C)

y_train = to_categorical(y_train1)
y_test = to_categorical(y_test1)

# reshape input to be [samples, time steps, features]
X_train = trainX
#X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
#X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))
X_test = testT

# Add a channels dimension
X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")
batch_size = 50



train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train1)).shuffle(1000000).batch(50)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test1)).batch(50)


class MyModel(Model):
    #
    def __init__(self):
        #
        super(MyModel, self).__init__()
        self.lstm1=LSTM(50,input_shape=(1, 76), return_sequences= True )
        self.drop1=Dropout(0.01)
        self.lstm2=LSTM(50,return_sequences=True)
        self.drop2=Dropout(0.01)
        self.lstm3=LSTM(50, return_sequences=False)
        self.drop3=Dropout(0.01)
        self.d1=Dense(9)
        self.activation1=Activation('softmax')

    def call(self, x):
        x = self.lstm1(x)
        x = self.drop1(x)
        x = self.lstm2(x)
        x = self.drop2(x)
        x = self.lstm3(x)
        x = self.drop3(x)
        x = self.d1(x)
        return self.activation1(x)

# Create an instance of the model
model = MyModel()

loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')


@tf.function
def train_step(traind, labels):
    
    with tf.GradientTape() as tape:
        #
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(traind, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

!mkdir -p saved_model
model.save('saved_model/my_model')
'''
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)
'''


Epoch 1, Loss: 0.9494781494140625, Accuracy: 76.67962646484375, Test Loss: 0.9318342804908752, Test Accuracy: 76.76829528808594
Epoch 2, Loss: 0.9336981773376465, Accuracy: 76.76829528808594, Test Loss: 0.9351197481155396, Test Accuracy: 76.76829528808594
Epoch 3, Loss: 0.8240154385566711, Accuracy: 79.20560455322266, Test Loss: 0.5629697442054749, Test Accuracy: 85.00102233886719
Epoch 4, Loss: 0.47244513034820557, Accuracy: 86.58572387695312, Test Loss: 0.3714466691017151, Test Accuracy: 89.8392562866211
Epoch 5, Loss: 0.33613133430480957, Accuracy: 90.00749969482422, Test Loss: 0.29892587661743164, Test Accuracy: 91.06472778320312


子目录或文件 saved_model 已经存在。
处理: saved_model 时出错。


INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


"\nnew_model = tf.keras.models.load_model('saved_model/my_model')\n\n# Check its architecture\nnew_model.summary()\n# Evaluate the restored model\nloss, acc = new_model.evaluate(test_images, test_labels, verbose=2)\nprint('Restored model, accuracy: {:5.2f}%'.format(100 * acc))\n\nprint(new_model.predict(test_images).shape)\n"